In [1]:
import sys, os
import pathlib
import time
import importlib
import joblib
import pickle
from tqdm import tqdm

import numpy as np

from scipy.linalg import null_space
from scipy.sparse import csr_matrix

import matplotlib.pyplot as plt

import cv2

In [2]:
import pickle
import itertools
import open3d as o3d
import numpy as np
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
%load_ext autoreload
%autoreload 2

if not os.path.abspath("../") in sys.path:
    sys.path.append(os.path.abspath("../"))
    sys.path.append(os.path.abspath("../src"))

from src import normalization
from src import Camera, metashape_Camera
from src import camera_correspondence, FF_mat, epipole_angle, pair_and_key_gen, coll_dict_gen, pair_pt_gen
from src import FR_check, coordinate_dict_gen
from src import min_dist, tri, TDlines_gen, reprojection_gen, gen_support_dict, gen_support
from src import all_D, frag_list_fraged, all_sep

In [98]:
def support_th(temp_folder, sup_th):
    curve_fragment = []
    for tag in tags:
        if tag[1] == "R":
            continue

        with open(os.path.join(temp_folder, "{0}_{1}_{2}.TDlines".format(tag[0][0],tag[0][1],tag[1])), 'rb') as f:
            lines_list = pickle.load(f)
        #lines_list = TDlines[tag]

        with open(os.path.join(temp_folder, "{0}_{1}_{2}.support_dict".format(tag[0][0],tag[0][1],tag[1])), 'rb') as f:
            support_list, support_ac = pickle.load(f)
        #support_list, support_ac = support_dict[tag][0], support_dict[tag][1]
        tag_list = []
        for col, sup_col, ac_col in zip(lines_list, support_list, support_ac):
            col_curve_fragment = []
            for frag, sup, sup_ac in zip(col, sup_col, ac_col):
                if sup > sup_th:
                    frag = np.reshape(frag,(-1, 3))
                    frag = np.array([i for i,j in zip(frag, sup_ac >sup_th) if j])
                    col_curve_fragment.append(frag)
            tag_list.append(col_curve_fragment)
        curve_fragment.append(tag_list)
    return curve_fragment

def rotation_mat(angle):
    Rx = np.array([[1,0,0],
                 [0, np.cos(angle[0]), -np.sin(angle[0])],
                 [0, np.sin(angle[0]), np.cos(angle[0])]])

    Ry = np.array([[np.cos(angle[1]), 0, np.sin(angle[1])],
                 [0,1,0],
                 [-np.sin(angle[1]), 0, np.cos(angle[1])]])

    Rz = np.array([[np.cos(angle[2]), -np.sin(angle[2]), 0],
                 [np.sin(angle[2]), np.cos(angle[2]), 0],
                 [0,0,1]])
    return Rz@Rx@Ry

angle = [0, np.pi/2, np.pi/2] # pcd.rotate()
angle = [-np.pi/2, np.pi/2, np.pi/2]
R = rotation_mat(angle)
R_mirror = np.array([[-1,0,0],
                    [0,1,0],
                    [0,0,1]])


def curve_fragment_align(curve_fragment):
    cfs = [[] for i in range(8)]
    for cols in curve_fragment:
        for i in range(len(cols)):
            cfs[i] += cols[i]
    cfs = [list(map(lambda x: x*10@R@R_mirror, cf_label)) for cf_label in cfs]
    
    return cfs

def get_pts(mesh, edges, max_point_num=1000):
    vertices = np.asarray(mesh.vertices)
    edges = np.asarray(edges)
    dist = 0
    for i in range(len(vertices[edges])):
        start = vertices[edges][i][0]
        end = vertices[edges][i][1]
        dist_part = np.sqrt(np.sum(np.square(start - end)))
        dist += dist_part
    unit_dist = dist/max_point_num
    next_start_length = 0
    next_start_partation = 0
    pts_list = []
    for i in range(len(vertices[edges])):
        start = vertices[edges][i][0]
        end = vertices[edges][i][1]
        next_st_pt = start + (end - start)*next_start_partation
        dist_part = np.sqrt(np.sum(np.square(start - end)))
        dist_nx = np.sqrt(np.sum(np.square(next_st_pt - end)))
        pts_num = int(dist_nx/unit_dist)+1
        pts = next_st_pt + np.repeat(np.arange(pts_num),3).reshape((pts_num,3)) * (end - next_st_pt)/(pts_num-1)
        end_pts = pts[-1]
        next_start_length = pts_num*unit_dist - dist_part + next_start_length
        next_start_partation = next_start_length/dist_part
        pts_list.append(pts)

    pts_array = np.concatenate(pts_list)

    return pts_array

def mesh_contours_load(individual):
    mesh = o3d.io.read_triangle_mesh("postprocessing/polygon/quan_e_a/{}.ply".format(individual))
    edge_pts=[]
    for i in range(np.max(np.asarray(mesh.cluster_connected_triangles()[0]))+1):
        mesh = o3d.io.read_triangle_mesh("postprocessing/polygon/quan_e_a/{}.ply".format(individual))
        mesh.triangles = o3d.utility.Vector3iVector(np.asarray(mesh.triangles)[np.asarray(mesh.cluster_connected_triangles()[0])==i])
        #_, _, area = mesh.cluster_connected_triangles()
        edges = mesh.get_non_manifold_edges(allow_boundary_edges=False)
        pts_array = get_pts(mesh, edges)
        edge_pts.append(pts_array)
    edge_pts[5], edge_pts[6], edge_pts[7] = edge_pts[6], edge_pts[7], edge_pts[5]
    return edge_pts

def cf_evaluation_core(edge_pts_label, cfs_label, t_dist=0.1, t_part=0.5):
    rec_TF_list = []
    for i, cf in enumerate(cfs_label):
        edge_pt_bro = np.repeat(edge_pts_label, len(cf), axis=0).reshape(edge_pts_label.shape[0], len(cf), edge_pts_label.shape[1])
        all_dist = np.sqrt(np.sum(np.square(cf - edge_pt_bro), axis=2))
        min_dist_idx_rec = np.argmin(all_dist.T, axis=1)
        min_dist_idx_pre = np.argmin(all_dist, axis=1)
        if i == 0:
            temp_dist_pre = np.ones(min_dist_idx_pre.shape)*1000
        min_dist_rec = all_dist.T[np.arange(len(all_dist.T)), min_dist_idx_rec]
        min_dist_pre = all_dist[np.arange(len(all_dist)), min_dist_idx_pre]
        rec_TF = (np.sum(min_dist_rec<t_dist)/len(min_dist_rec))>t_part
        rec_TF_list.append(rec_TF)
        temp_dist_pre[np.where(temp_dist_pre>min_dist_pre)[0]] = min_dist_pre[np.where(temp_dist_pre>min_dist_pre)[0]]
    recall = sum(rec_TF_list)/len(rec_TF_list)
    precision = np.sum(temp_dist_pre<t_dist)/len(temp_dist_pre)
    return precision, recall

def cf_evaluation(edge_pts, cfs):
    leaves_precision = []
    leaves_recall = []
    for edge_pts_label, cfs_label in zip(edge_pts, cfs):
        precision, recall = np.nan, np.nan
        if len(cfs_label) != 0:
            precision, recall = cf_evaluation_core(edge_pts_label, cfs_label)
    leaves_precision.append(precision)
    leaves_recall.append(recall)
    return leaves_precision, leaves_recall

def evaluation(individual, img_num, temp_folder):
    precision_list = []
    recall_list = []
    edge_pts = mesh_contours_load(individual)
    for sup_th in range(img_num+1):
        curve_fragment = support_th(temp_folder, sup_th)
        cfs = curve_fragment_align(curve_fragment)
        precision, recall = cf_evaluation(edge_pts, cfs)
        precision_list.append(precision)
        recall_list.append(recall)
    return precision_list, recall_list

In [99]:
img_num = 128
individual = "q_1_a"
temp_folder = "temp"

In [100]:
cam_list = [Camera(i) for i in range(img_num)]
for i in range(len(cam_list)):
    cam_list[i].img_load("images/quan_equidistant/{}/{}".format(individual, str(img_num)))
    cam_list[i].contour_extraction()
    cam_list[i].label_load("preprocessing/labels/q_e_l_noise_3mm/{}/{}_{}_label.pickle".format(individual, individual, str(img_num)))
    cam_list[i].correspondence_contour()
    cam_list[i].para_load("view_mats/q_e_noise_3mm/view_mat_{}/{}.csv".format(str(img_num), str(i)))

In [101]:
kernel = np.ones((3,3),np.uint8)
w = 5
for cnum in range(len(cam_list)):
    sparse_mats_list = []
    for j in range(len(cam_list[cnum].contour_list)):
        new_img = np.zeros((cam_list[0].img[0].shape[0],cam_list[0].img[0].shape[1]),dtype=np.uint8)
        for i in range(len(cam_list[cnum].contour_list[j])):
            curve = cam_list[cnum].contour_list[j][i][~np.isnan(cam_list[cnum].contour_list[j][i])].reshape((-1,2)).astype(int)
            new_img[curve[:,1],curve[:,0]]=True
        dilation = cv2.dilate(new_img, kernel, iterations = w)
        new_img = csr_matrix(dilation, dtype=np.uint8)
        sparse_mats_list.append(new_img)
    cam_list[cnum].contour_img = sparse_mats_list

In [102]:
temp_path = pathlib.Path(temp_folder)
tags_path = list(temp_path.glob("*.TDlines"))

tags = []
for tag_path in tags_path:
    tag_sp = tag_path.stem.split("_")
    tag_arr = ((int(tag_sp[0]), int(tag_sp[1])), tag_sp[2])
    tags.append(tag_arr)